In [1]:
# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# # LightAutoML presets, task and report generation
# from lightautoml.automl.presets.tabular_presets import TabularAutoML
# from lightautoml.tasks import Task

In [2]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 1500 # equal to 25 minutes
TARGET_NAME = 'final_price'

In [3]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [4]:
INPUT_DIR = './Kaggle_0/'

In [23]:
train_data = pd.read_csv(INPUT_DIR + 'train_data.csv')
print(train_data.shape)
train_data.head(3)

(35000, 15)


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0


In [24]:
test_data = pd.read_csv(INPUT_DIR + 'test_data.csv')
print(test_data.shape)
test_data.head(3)

(10697, 14)


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type
0,35000,TOYOTA,Prius,Hatchback,323733,2012,Automatic,4/5,Left wheel,Grey,Black,JTDKN3DU6C5439638,1,For Sale
1,35001,HYUNDAI,Elantra,Sedan,112000,2013,Tiptronic,4/5,Left wheel,Grey,Black,SURATSHIA,1,For Sale
2,35002,LEXUS,NX 300,Jeep,16920,2018,Automatic,NaN,Left wheel,Brown,NaN,JTJYARBZ5J2104521,1,For Sale


In [7]:
submission = pd.read_csv(INPUT_DIR + 'sample_submission.csv')
print(submission.shape)
submission.head(3)

(10697, 2)


,row_ID,final_price
0,35000,0
1,35001,0
2,35002,0


In [ ]:
train_data[train_data['vehicle_model'] == 'C 220']

In [12]:
train_data.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price,car_wearout_ratio
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,JTN,0,For Sale,3650.0,0.939850
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,4JG20,0,For Sale,6800.0,3.401361
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,MHTC6,1,For Sale,6300.0,5.376344
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,KMHS2,1,For Sale,14488.0,1.237298
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,TNKHM,1,For Sale,5000.0,17.590150


In [25]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   row_ID                  35000 non-null  int64  
 1   vehicle_manufacturer    34999 non-null  object 
 2   vehicle_model           34993 non-null  object 
 3   vehicle_category        34999 non-null  object 
 4   current_mileage         35000 non-null  int64  
 5   vehicle_year            35000 non-null  int64  
 6   vehicle_gearbox_type    34999 non-null  object 
 7   doors_cnt               34255 non-null  object 
 8   wheels                  34999 non-null  object 
 9   vehicle_color           34599 non-null  object 
 10  vehicle_interior_color  28282 non-null  object 
 11  car_vin                 11488 non-null  object 
 12  car_leather_interior    35000 non-null  int64  
 13  deal_type               35000 non-null  object 
 14  final_price             35000 non-null

In [13]:
# def append_wmi(wmis, vendor, car_vin):
#     """ This function appends new element in dictionary
#         if not exist

#     Args:
#         wmis (dictionary of the lists of the codes) like {'manufacturer": ['JTG', '1T4', ... 'JTD']} 
#         vendor (text string): name of the vehicle manufacturer
#         car_vin (text string): serial of the numbers and symbols (max 17)
#     """
#     if car_vin in ['nan', 'NaN']:
#         return
#     wmi = car_vin[:3]
  
#     if vendor not in wmis.keys():
#         wmis.update({vendor: list()})
#         wmis[vendor].append(wmi)
    
#     elif wmi not in wmis[vendor]:
#         wmis[vendor].append(wmi)
      
#     return


# def append_vds(vdss, model, car_vin):
#     """ This function appends new element in dictionary
#         if not exist

#     Args:
#         vdcs (dictionary of the lists of the codes) like {'model": ['XC', 'SD', ... 'SC']}
#         model (text string): name of the vehicle model
#         car_vin (text string): serial of the numbers and symbols (max 17)
#     """
#     if car_vin in ['nan', 'NaN'] or not len(model):
#         return
#     vds = car_vin[3:5]
  
#     if model not in vdss.keys():
#         vdss.update({model: list()})
#         vdss[model].append(vds)
     
#     elif vds not in vdss[model]:
#         vdss[model].append(vds)
       
#     return


# def get_vin_parts(df):
#     """ This function collect existing vin-codes into dictionary
#         for future implementations

#     Args:
#         df (pandas.DataFrame): The source data that we use for collecting
#     Returns:
#         wmis_dict (dictionary of the lists of the codes) like {'manufacturer": ['JTG', '1T4', ... 'JTD']} 
#         vdcs_dict (dictionary of the lists of the codes) like {'model": ['XC', 'SD', ... 'SC']}
#     """
#     # Create empty dictionaries
#     wmi_dict = dict()
#     vds_dict = dict()
    
#     # Go along the rows by indexes
#     for row in df.index:
#         car_vin = str(df['car_vin'].iloc[row])

#         if len(car_vin) > 3:
            
#             vendor = str(df['vehicle_manufacturer'].iloc[row])
#             # Get World Manufacturers Identification (wmi)
#             append_wmi(wmi_dict, vendor, car_vin)
            
#             model = str(df['vehicle_model'].iloc[row])
#             # Get Vehicle Description Section (vds)
#             append_vds(vds_dict, model, car_vin)
            
#     return wmi_dict, vds_dict


# def set_vin(wmis, vdss, df):
#     """ This function set vin-codes for empty cells
#         using vendor and model information

#     Args:
#         wmis (dictionary of the lists of the codes) like {'manufacturer": ['JTG', '1T4', ... 'JTD']} 
#         vdcs (dictionary of the lists of the codes) like {'model": ['XC', 'SD', ... 'SC']}
#         df (pandas.DataFrame): The source dataframe

#     Returns:
#         pandas.DataFrame: output dataframe
#     """
#     for row in df.index:
#         car_vin = str(df['car_vin'].iloc[row])
        
#         if car_vin in ['nan', 'NaN']:
#             vendor = str(df['vehicle_manufacturer'].iloc[row])
#             model = str(df['vehicle_model'].iloc[row])
            
#             if vendor in wmis.keys():
#                 car_vin = wmis[vendor][0]
#             else:
#                 car_vin = 'nan'
#             if model in vdss.keys():
#                 car_vin += vdss[model][0]
#         else:
#             car_vin = car_vin[1:6]
#         df['car_vin'].iloc[row] = car_vin
                
#     return df

In [26]:
def append_wmi_vds(vins, vendor, model, body_type,car_vin):
    """ This function appends new element in dictionary
        if not exist

    Args:
        wmis (dictionary of the lists of the codes) like {'manufacturer": ['JTG', '1T4', ... 'JTD']} 
        vendor (text string): name of the vehicle manufacturer
        car_vin (text string): serial of the numbers and symbols (max 17)
    """
    if car_vin in ['nan', 'NaN']:
        return
    vin_part = car_vin[:7]
    vehicle = vendor + ' ' + model + ' ' + body_type
  
    if vehicle not in vins.keys():
        vins.update({vehicle: list()})
        vins[vehicle].append(vin_part)
    
    elif vin_part not in vins[vehicle]:
        vins[vehicle].append(vin_part)
      
    return



def get_vin_parts(df):
    """ This function collect existing vin-codes into dictionary
        for future implementations

    Args:
        df (pandas.DataFrame): The source data that we use for collecting
    Returns:
        wmis_dict (dictionary of the lists of the codes) like {'manufacturer": ['JTG', '1T4', ... 'JTD']} 
        vdcs_dict (dictionary of the lists of the codes) like {'model": ['XC', 'SD', ... 'SC']}
    """
    # Create empty dictionary
    vin_dict = dict()
    
    # Go along the rows by indexes
    for row in df.index:
        car_vin = str(df['car_vin'].iloc[row])

        if len(car_vin) > 3:
            vendor = str(df['vehicle_manufacturer'].iloc[row])
            model = str(df['vehicle_model'].iloc[row])
            body_type = str(df['vehicle_category'].iloc[row])
            
            # Get World Manufacturers Identification (wmi) + Vehicle Description Section (vds)
            append_wmi_vds(vin_dict, vendor, model, body_type, car_vin)
            
    return vin_dict


def set_vin_parts(vins, df):
    """ This function set vin-codes for empty cells
        using vendor and model information

    Args:
        wmis (dictionary of the lists of the codes) like {'manufacturer": ['JTG', '1T4', ... 'JTD']} 
        vdcs (dictionary of the lists of the codes) like {'model": ['XC', 'SD', ... 'SC']}
        df (pandas.DataFrame): The source dataframe

    Returns:
        pandas.DataFrame: output dataframe
    """
    for row in df.index:
        car_vin = str(df['car_vin'].iloc[row])
        
        if car_vin in ['nan', 'NaN']:
            vendor = str(df['vehicle_manufacturer'].iloc[row])
            model = str(df['vehicle_model'].iloc[row])
            body_type = str(df['vehicle_category'].iloc[row])
            vehicle = vendor + ' ' + model + ' ' + body_type
            if vehicle in vins.keys():
                car_vin = vins[vehicle][0]
            else:
                car_vin = 'nan'
        else:
            car_vin = car_vin#[1:6]
        df['car_vin'].iloc[row] = car_vin
                
    return df

In [27]:

#import category_encoders as ce

def create_expert_feats(data):
    
    # # 1 ++ 
    # def modify_car_vin(car_vin):
    #     car_vin = str(car_vin)
    #     if car_vin not in ['nan', 'Nan']:
    #         # Remove excess spaces
    #         car_vin = ''.join(car_vin.split(' '))
    #     else:
    #         return car_vin
    #     if len(car_vin) < 11:
    #         return car_vin
    #     else:
    #         return car_vin[:11]
        
    # data['car_vin'] =  data['car_vin'].apply(modify_car_vin)
    
    data['car_vin'] =  data['car_vin'].apply(lambda x: ''.join(str(x).split(' ')))
    
    data['vehicle_manufacturer'] =  data['vehicle_manufacturer'].apply(lambda x: ''.join(str(x).split(' ')))
  
    data['vehicle_model'] =  data['vehicle_model'].apply(lambda x: ''.join(str(x).split(' ')))
    
    data['vehicle_category'] =  data['vehicle_category'].apply(lambda x: ''.join(str(x).split(' ')))
    
    display(data.head(7))
    
    vin_dict = get_vin_parts(data)
    display(vin_dict)
    data = set_vin_parts(vin_dict, data)

    # 3 +++
    data['car_wearout_ratio'] = 1e6/((2022 - data['vehicle_year'])*data['current_mileage'])
    
    # 4
#     bin_encoder = ce.BinaryEncoder(cols=['vehicle_category'])
#     type_bin = bin_encoder.fit_transform(data['vehicle_category'])
#     data = pd.concat([data, type_bin], axis=1)
    
#     bin_encoder = ce.BinaryEncoder(cols=['vehicle_gearbox_type'])
#     type_bin = bin_encoder.fit_transform(data['vehicle_gearbox_type'])
#     data = pd.concat([data, type_bin], axis=1)
#     # 5
#     dupl_columns = list(data.columns)
#     dupl_columns.remove('row_ID')
    
#     mask = data.duplicated(subset=dupl_columns)
#     car_duplicates = data[mask]
#     print(f'Число найденных дубликатов: {car_duplicates.shape[0]}')
#     data = data.drop_duplicates(subset=dupl_columns)
#     print(f'Результирующее число записей: {data.shape[0]}')

    display(data.head(5))
    return data

In [28]:
train_data = create_expert_feats(train_data)
#test_data = create_expert_feats(test_data)

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aquas,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,nan,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,nan,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0
3,3,HYUNDAI,SantaFE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,nan,1,For Sale,14488.0
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0
5,5,MITSUBISHI,Delica,Jeep,149000,2003,Automatic,4/5,Right-hand drive,Silver,Black,nan,0,For Sale,20.0
6,6,HYUNDAI,Tucson,Jeep,173600,2013,Automatic,4/5,Left wheel,Silver,Black,nan,1,For Sale,7632.0


{'HYUNDAI Veloster Hatchback': ['KMHTC6A',
  'KMHT36A',
  'KMHTG6A',
  'VIN:KMH',
  'KMHTH6A'],
 'TOYOTA CHR Jeep': ['JTNKHMB', 'NMTKHMB'],
 'MERCEDES-BENZ GLE350 Sedan': ['4JGDA5J',
  '4JGFB4J',
  '4JGED6E',
  '4JGDA6E',
  '4JGDA5G'],
 'NISSAN Juke Jeep': ['JN8AF5M', 'JN8DF5M'],
 'LEXUS GX470 Jeep': ['JTJBT20', 'JTJJM7F', 'JTJBM7F'],
 'AUDI Q5 Jeep': ['WA1VFCF',
  'WA1ANAF',
  'WA1D7AF',
  'WA1LFAF',
  'WA1DKAF',
  'WA1WKAF',
  'WA1L2AF',
  'WA1C8AF',
  'WA1DGAF',
  'WA1LFCF',
  'WA1BNAF',
  'WA1C2AF',
  'WA1DGCF'],
 'BMW X5 Jeep': ['5UXZW0C',
  '5UXKS4C',
  '5UXZV8C',
  '5UXZV4C',
  '5UXKR0C',
  '5UXJU2C',
  '5UXKR2C',
  '5UXFA53',
  '5UXFF0C',
  '5UXFF03',
  '4444',
  '5UXKT0C',
  '5UXZV4G',
  '5UXKR6C',
  'WBAFB71',
  'LT75116',
  '5784336',
  '5UXCR6C',
  'SURATEB',
  'SURATSH',
  '5UXFE8C',
  '5YMGY0C',
  'WBALSO1',
  '5UXFE43',
  'FFI64RF',
  'DFHHSHJ'],
 'TOYOTA Camry Sedan': ['4T1BD1F',
  'JTNB11H',
  '4T4BF1F',
  '4T1BD1E',
  '4T1BF1F',
  '4T1BK1F',
  '4T1B31H',
  '4T1B11H',


/home/freeradical/anaconda3/envs/sf/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price,car_wearout_ratio
0,0,TOYOTA,Aquas,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,nan,0,For Sale,3650.0,0.939850
1,1,MERCEDES-BENZ,C220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,WDB2030,0,For Sale,6800.0,3.401361
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0,5.376344
3,3,HYUNDAI,SantaFE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,5NMS2CA,1,For Sale,14488.0,1.237298
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0,17.590150


In [ ]:
train_data['car_vin'].value_counts()

In [11]:
%%time
import category_encoders as ce

def create_expert_feats(data):
    
    # 1 ++ 
    def modify_car_vin(car_vin):
        if car_vin != np.nan:
            car_vin = str(car_vin)
        else:
            return car_vin
        if len(car_vin) < 11:
            return car_vin
        else:
            return car_vin[:11]
    data['car_vin'] =  data['car_vin'].apply(modify_car_vin)
    
    # 2 +-0
    #создаем словарь имя столбца: число(признак) на который надо заменить пропуски
#     values = {
#         'doors_cnt': data['doors_cnt'].mode()[0],
#         'vehicle_color': data['vehicle_color'].mode()[0],
#         'vehicle_interior_color': data['vehicle_interior_color'].mode()[0]
#     }
#     #заполняем пропуски в соответствии с заявленным словарем
#     data = data.fillna(values)

    # 3 +++
    data['car_wearout_ratio'] = 1e6/((2022 - data['vehicle_year'])*data['current_mileage'])
    
    # 4
#     bin_encoder = ce.BinaryEncoder(cols=['vehicle_category'])
#     type_bin = bin_encoder.fit_transform(data['vehicle_category'])
#     data = pd.concat([data, type_bin], axis=1)
    
#     bin_encoder = ce.BinaryEncoder(cols=['vehicle_gearbox_type'])
#     type_bin = bin_encoder.fit_transform(data['vehicle_gearbox_type'])
#     data = pd.concat([data, type_bin], axis=1)

    display(data.head(3))
    return data
   
train_data = create_expert_feats(train_data)
test_data = create_expert_feats(test_data)

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price,car_wearout_ratio
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,nan,0,For Sale,3650.0,0.939850
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,nan,0,For Sale,6800.0,3.401361
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3G,1,For Sale,6300.0,5.376344


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,car_wearout_ratio
0,35000,TOYOTA,Prius,Hatchback,323733,2012,Automatic,4/5,Left wheel,Grey,Black,JTDKN3DU6C,1,For Sale,0.308897
1,35001,HYUNDAI,Elantra,Sedan,112000,2013,Tiptronic,4/5,Left wheel,Grey,Black,SURATSHIA,1,For Sale,0.992063
2,35002,LEXUS,NX 300,Jeep,16920,2018,Automatic,NaN,Left wheel,Brown,NaN,JTJYARBZ5J,1,For Sale,14.775414


CPU times: user 72.6 ms, sys: 8.06 ms, total: 80.6 ms
Wall time: 113 ms


In [ ]:
tr_data, te_data = train_test_split(
    train_data, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)

print(f'Data splitted. Parts sizes: tr_data = {tr_data.shape}, te_data = {te_data.shape}')

tr_data.head()

In [ ]:
task = Task('reg', loss = 'mae', metric = 'mae')

In [ ]:
roles = {
    'target': TARGET_NAME,
    'drop': ['row_ID']
}